# **Cluster Management**

This notebooke is to assist with managing the your cluster.

## Available Machines

This is from the canvas announcement.

### Freddy

**Ports:** [30785, 30799] 

1. columbia 
2. columbus-oh 
3. concord 
4. denver 
5. des-moines 
6. dover 
7. frankfort 
8. harrisburg 
9. hartford 
10. helena 
11. honolulu 
12. indianapolis 

### Eric

**Ports:** [30181]

1. hartford
2. helena
3. honolulu
4. indianapolis
5. des-moines
6. dover
7. frankfort
8. harrisburg

In [1]:
# Specify your user name
username = 'flarrieu'
# username = 'ebmartin'

In [2]:
# Current Machines and port in use

try : 
    with open(f'/s/chopin/a/grad/{username}/bigdata_configs/machines_and_ports.txt') as f: 
        s = f.read()
        print(s)
except FileNotFoundError:
    print('File not found')

try :
    with open(f'/s/chopin/l/grad/{username}/bigdata_configs/machines_and_ports.txt') as f: 
        s = f.read()
        print(s)
except FileNotFoundError:
    print('File not found')

File not found
hartford
helena
honolulu
indianapolis
des-moines
dover
frankfort
harrisburg
30181



In [3]:
# Machine Roles After Cluster Setup

try:
    with open(f'/s/chopin/a/grad/{username}/bigdata_configs/README.md', 'r') as file:
        lines = file.readlines()
except FileNotFoundError:
    print('File not found')

try:
    with open(f'/s/chopin/l/grad/{username}/bigdata_configs/README.md', 'r') as file:
        lines = file.readlines()
except FileNotFoundError:
    print('File not found')
    
machine_roles = lines[-8:]

for machine in machine_roles:
    print(machine)
    if "namenode" in machine and not "secondary" in machine:
        namenode = machine.split(" ")[2]
    if "resource manager" in machine:
        resource_manager = machine.split(" ")[3]

File not found
- namenode: hartford

- secondary namenode: helena

- resource manager: honolulu

- datanode1: indianapolis

- datanode2: des-moines

- datanode3: dover

- datanode4: frankfort

- datanode5: harrisburg


# **Current Cluster Status**

In [4]:
%%bash
# Print the current status of the cluster
# You will need to set the following in your ~/.bashrc: 
#   chmod +x /s/chopin/a/grad/{username}/hadoopConf/monitor.sh
#   export MONITOR_CLUSTER="/s/chopin/a/grad/{username}/hadoopConf/monitor.sh"

source ~/.bashrc
$MONITOR_CLUSTER

/usr/share/Modules/init/bash: line 113: unalias: pip3: not found
/usr/share/Modules/init/bash: line 114: unalias: pip: not found


hartford
273880 QuorumPeerMain
1705892 Jps
helena
927550 UIServer
927549 Nimbus
799188 Jps
honolulu
3292581 Jps
indianapolis
1075349 Jps
161556 Supervisor
des-moines
153094 Jps
dover
379412 Supervisor
3488382 Jps
frankfort
166968 Supervisor
841857 Jps
harrisburg
3146355 Jps
191435 Supervisor


## **Starting DFS and Yarn**

In [8]:
%pip install paramiko

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/p

In [9]:
import os
import paramiko

# You will need to set the following in your ~/.bashrc: 
#   export PASSWORD="your password"

password = os.getenv("PASSWORD")

if not password:
    print("Error: You need to set your password in you bashrc.")

Error: You need to set your password in you bashrc.


In [10]:
# Start DFS
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/start-dfs.sh') # Run start-dfs.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

NameError: name 'ssh' is not defined

In [ ]:
# Start Yarn
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(resource_manager.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/start-yarn.sh') # Run start-yarn.sh on resource manager
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

Starting resourcemanager
Starting nodemanagers
frankfort: nodemanager is running as process 418169.  Stop it first and ensure /tmp/hadoop-flarrieu-nodemanager.pid file is empty before retry.
dover: nodemanager is running as process 2340746.  Stop it first and ensure /tmp/hadoop-flarrieu-nodemanager.pid file is empty before retry.
harrisburg: nodemanager is running as process 1620452.  Stop it first and ensure /tmp/hadoop-flarrieu-nodemanager.pid file is empty before retry.
des-moines: nodemanager is running as process 441463.  Stop it first and ensure /tmp/hadoop-flarrieu-nodemanager.pid file is empty before retry.
denver: nodemanager is running as process 1399813.  Stop it first and ensure /tmp/hadoop-flarrieu-nodemanager.pid file is empty before retry.

resourcemanager is running as process 703039.  Stop it first and ensure /tmp/hadoop-flarrieu-resourcemanager.pid file is empty before retry.



## **Stopping Master and Workers**

In [ ]:
# Stop DFS
if password:
    sure = input("Are you sure: y or n?")
    if sure == "y":
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(namenode.strip(), username=username, password=password)
        stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/stop-dfs.sh') # Run stop-dfs.sh on namenode
        print(stdout.read().decode())
        print(stderr.read().decode())

ssh.close()    

In [ ]:
# Start Yarn
if password:
    sure = input("Are you sure: y or n?")
    if sure == "y":
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(resource_manager.strip(), username=username, password=password)
        stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/stop-yarn.sh') # Run start-yarn.sh on resource manager
        print(stdout.read().decode())
        print(stderr.read().decode())

ssh.close()    

# **Starting Master and Workers**

In [ ]:
# Start master
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/start-master.sh') # Run start-master.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

org.apache.spark.deploy.master.Master running as process 23244.  Stop it first.




In [ ]:
# Start workers
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/start-workers.sh') # Run start-master.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

frankfort: org.apache.spark.deploy.worker.Worker running as process 419651.  Stop it first.
frankfort: org.apache.spark.deploy.worker.Worker running as process 419776.  Stop it first.
dover: org.apache.spark.deploy.worker.Worker running as process 2344384.  Stop it first.
dover: org.apache.spark.deploy.worker.Worker running as process 2344532.  Stop it first.
harrisburg: org.apache.spark.deploy.worker.Worker running as process 1626417.  Stop it first.
harrisburg: org.apache.spark.deploy.worker.Worker running as process 1626582.  Stop it first.
denver: org.apache.spark.deploy.worker.Worker running as process 1405900.  Stop it first.
denver: org.apache.spark.deploy.worker.Worker running as process 1406144.  Stop it first.
des-moines: org.apache.spark.deploy.worker.Worker running as process 442949.  Stop it first.
des-moines: org.apache.spark.deploy.worker.Worker running as process 443072.  Stop it first.




# **Clients**

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
# Parsing .xml
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    properties = {}
    for property in root.findall('property'):
        name = property.find('name').text
        value = property.find('value').text
        properties[name] = value
    return properties

In [ ]:
# Parsing .conf
def parse_spark_conf(file_path):
    properties = {}
    with open(file_path, 'r') as file:
        for line in file:
            if 'spark.master' in line and not "#" in line:
                key, value = line.split(" ", 1)
                properties[key.strip()] = value.strip()
                
    return properties

In [ ]:
# Parsing .env
def parse_spark_env(file_path):
    properties = {}
    with open(file_path, 'r') as file:
        for line in file:
            if 'export SPARK_MASTER_WEBUI_PORT' in line:
                key, value = line.split("=", 1)
                properties['port'] = value.split(" ")[0]
                pass
    return properties

In [ ]:
hdfs_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/hdfs-site.xml"
core_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/core-site.xml"
yarn_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/yarn-site.xml"
spark_conf_path = "/s/chopin/a/grad/flarrieu/sparkConf/spark-defaults.conf"
spark_env_path = "/s/chopin/a/grad/flarrieu/sparkConf/spark-env.sh"

hdfs_properties = parse_xml(hdfs_site_path)
core_properties = parse_xml(core_site_path)
yarn_properties = parse_xml(yarn_site_path)
spark_properties = parse_spark_conf(spark_conf_path)
spark_env_properties = parse_spark_env(spark_env_path)

print("HDFS Client")
print("http://" + hdfs_properties.get("dfs.namenode.http-address"))

print("Core Site")
print(core_properties.get('fs.default.name'))

print("Resource Management Client")
print("http://" + resource_manager.strip() + ".cs.colostate.edu:" + yarn_properties.get('yarn.resourcemanager.webapp.address').split(":")[1])

print("Spark")
print(spark_properties.get('spark.master'))

print("Spark Client")
print("http://" + namenode.strip() + ".cs.colostate.edu:" + spark_env_properties.get("port"))

HDFS Client
http://columbus-oh.cs.colostate.edu:30786
Core Site
hdfs://columbus-oh.cs.colostate.edu:30785
Resource Management Client
http://columbia.cs.colostate.edu:30798
Spark
spark://columbus-oh.cs.colostate.edu:30800
Spark Client
http://columbus-oh.cs.colostate.edu:30801
